In [1]:
#import libraries
import numpy as np
import requests
import pandas as pd
import csv
#!conda install beautifulsoup4

In [2]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup
import xml

# Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas datafram

In [4]:
#define URL to webscape
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'html')
#makes it east to read
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xh6LeApAIDEAACUs1LEAAACJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

In [5]:
#the table data we want is here
table = soup.find('table',{'class':'wikitable sortable'})
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [6]:
#looks as though the values of td are what we want so to find all listed as td 
links = table.find_all('td')
links

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Manor" title="Lawrence Manor">Lawrence Manor</a>
 </td>, <td>M7A</td>, <td>

In [7]:
#with this cleaner version we can finally start to scrape what we want
#creat lists of what we want from the table

postecode = []
borough = []
neighborhood = []

In [8]:
#scrapes values of td for only text and assigns them to there respective areas 
for i in range(0, len(links), 3):
    postecode.append(links[i].find(text=True))
    borough.append(links[i+1].find(text=True))
    neighborhood.append(links[i+2].find(text=True).rstrip())

In [9]:
#puts it into a data frame
df_postalfields = pd.DataFrame(data=[postecode, borough, neighborhood]).transpose()
df_postalfields.columns = ['Postecode', 'Borough', 'Neighborhood']

In [10]:
df_postalfields.head(20)

,Postecode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [11]:
#Clean and remove postal codeds under borough that are labled not assigned

#make not assigned = missing value
df_postalfields['Borough'].replace('Not assigned', np.nan, inplace=True)
#remove missing values
df_postalfields.dropna(subset=['Borough'], axis=0, inplace=True)
df_postalfields.head(20)

,Postecode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [12]:
#combine similar neighborhoods with poste codes using comma

#Saves postecode and borough and shows combined neighborhoods in new data frame
df_pf = df_postalfields.groupby(['Postecode','Borough'])['Neighborhood'].apply(','.join).reset_index()

#new columns for data frame
df_pf.columns=['Postecode','Borough','Neighborhood']
df_pf

,Postecode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [13]:
#Changes values in Neighborhood from not assigned to value listed in borough
df_pf['Neighborhood'].replace('Not assigned', 'Borough', inplace=True)
df_pf

,Postecode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [14]:
df_pf.shape

(103, 3)

# Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [15]:
geo_data = ('http://cocl.us/Geospatial_data')
df_geo_data = pd.read_csv(geo_data)
df_geo_data.columns = ['Postecode','Latitude','Logitude']
df_geo_data.head()

,Postecode,Latitude,Logitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merge the 2 tables together so that it includes longitude and latitude

In [16]:
#add latitude and longitude to previous table
df_pf_geo = pd.merge(df_pf, df_geo_data, on='Postecode')
df_pf_geo

,Postecode,Borough,Neighborhood,Latitude,Logitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


# Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto

In [17]:
#folium
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import folium
import json
import matplotlib as mp
from sklearn.cluster import KMeans

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [18]:
#Map of toronto using latitude and longitude values
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="T_On")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("the geographical coordinate of Toronto is {}, {}".format(latitude,longitude))

the geographical coordinate of Toronto is 43.653963, -79.387207


# Create a map of Toronto with Boroughs superimposed on top

In [19]:
#create map of toronto with boroughs data

toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, post, borough, neigh in zip(df_pf_geo['Latitude'], df_pf_geo['Logitude'], df_pf_geo['Postecode'], df_pf_geo['Borough'], df_pf_geo['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
    
toronto_map

# Reduce the number of Boroughs to explore to reduce number of calls to FourSquare API. So we will only look at boroughs with Toronto in their names 

In [20]:
toronto_borough = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
df_pf_geo_tor = df_pf_geo[df_pf_geo['Borough'].isin(toronto_borough)].reset_index(drop=True)
print(df_pf_geo_tor.shape)
df_pf_geo_tor.head(40)

(39, 5)


,Postecode,Borough,Neighborhood,Latitude,Logitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [21]:
toronto_borough_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, post, borough, neigh in zip(df_pf_geo_tor['Latitude'], df_pf_geo_tor['Logitude'], df_pf_geo_tor['Postecode'], df_pf_geo_tor['Borough'], df_pf_geo_tor['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_borough_map)

toronto_borough_map

# Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = '1YYOWV1IOQDCLE4GDZKQLOJPRKDQRJ3W45BLB4SXDKNUSNJ2' # your Foursquare ID
CLIENT_SECRET = 'TT2LTZDS4FTSGT2IB2E0FLSVY1EAMHNHWLPT5S1KACWDI5OF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1YYOWV1IOQDCLE4GDZKQLOJPRKDQRJ3W45BLB4SXDKNUSNJ2
CLIENT_SECRET:TT2LTZDS4FTSGT2IB2E0FLSVY1EAMHNHWLPT5S1KACWDI5OF


In [23]:
#function that extracts the category of the venue
radius = 500
LIMIT = 100

venues_list = []

for lat,lng, post, borough, neighborhood in zip(df_pf_geo_tor['Latitude'],df_pf_geo_tor['Logitude'],df_pf_geo_tor['Postecode'],df_pf_geo_tor['Borough'],df_pf_geo_tor['Neighborhood']):
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for v in results:
        venues_list.append((
                post,
                borough,
                neighborhood,
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name'])) 

# Clean json and put into pandas data frame 

In [24]:
df_venues = pd.DataFrame(venues_list)
df_venues.columns = ['Postecode', 
                     'Borough', 
                     'Neighborhood',
                     'Borough Latitude', 
                     'Borough Longitude', 
                     'Venue Name',
                     'Venue Latitude',
                     'Venue Logitude',
                     'Venue Category']
print(df_venues.shape)
df_venues.head()

(1702, 9)


,Postecode,Borough,Neighborhood,Borough Latitude,Borough Longitude,Venue Name,Venue Latitude,Venue Logitude,Venue Category
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


# Check how many venues were returned for each Borough

In [25]:
df_venues.groupby('Borough').count()

,Postecode,Neighborhood,Borough Latitude,Borough Longitude,Venue Name,Venue Latitude,Venue Logitude,Venue Category
Borough,,,,,,,,
Central Toronto,110,110,110,110,110,110,110,110
Downtown Toronto,1304,1304,1304,1304,1304,1304,1304,1304
East Toronto,123,123,123,123,123,123,123,123
West Toronto,165,165,165,165,165,165,165,165


# How many unique categories are curated from all returned venues 

In [26]:
print ('There are {} unique categories'.format(len(df_venues['Venue Category'].unique())))

There are 232 unique categories


# Analyze each borough

In [27]:
#one hot encoding
Toronto_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix = "", prefix_sep = "")

#add neighborhood column back to dataframe
Toronto_onehot['Borough'] = df_venues['Borough']

#move Borough to first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,East Toronto,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Group rows by borough and take mean of frequency of occurence in each category

In [28]:
Toronto_grouped = Toronto_onehot.groupby('Borough').mean().reset_index()
Toronto_grouped

,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,...,0.009091,0.009091,0.000000,0.009091,0.000000,0.009091,0.000000,0.000000,0.000000,0.009091
1,Downtown Toronto,0.000767,0.000767,0.000767,0.001534,0.001534,0.001534,0.009969,0.001534,0.003834,...,0.000767,0.000767,0.002301,0.012270,0.001534,0.005368,0.006135,0.000767,0.001534,0.002301
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.000000,...,0.000000,0.016260,0.000000,0.000000,0.000000,0.000000,0.008130,0.000000,0.000000,0.016260
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.012121,0.000000,0.012121,0.006061,0.000000,0.000000,0.018182


In [29]:
Toronto_grouped.shape

(4, 233)

# Each Borough wigh top 5 most common venues

In [30]:
num_top_venues = 5

for hood in Toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['Venue', 'Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency':2})
    print(temp.sort_values('Frequency', ascending = False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
              Venue  Frequency
0       Coffee Shop       0.07
1              Park       0.06
2    Sandwich Place       0.06
3              Café       0.05
4  Sushi Restaurant       0.04


----Downtown Toronto----
         Venue  Frequency
0  Coffee Shop       0.10
1         Café       0.05
2        Hotel       0.03
3   Restaurant       0.03
4          Bar       0.02


----East Toronto----
                Venue  Frequency
0    Greek Restaurant       0.07
1         Coffee Shop       0.07
2  Italian Restaurant       0.05
3             Brewery       0.04
4      Ice Cream Shop       0.04


----West Toronto----
                Venue  Frequency
0                 Bar       0.08
1                Café       0.07
2         Coffee Shop       0.05
3          Restaurant       0.04
4  Italian Restaurant       0.04




In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Borough_venues_sorted = pd.DataFrame(columns=columns)
Borough_venues_sorted['Borough'] = Toronto_grouped['Borough']

for ind in np.arange(Toronto_grouped.shape[0]):
    Borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Park,Sandwich Place,Café,Restaurant,Sushi Restaurant,Pizza Place,Dessert Shop,Clothing Store,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Seafood Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Café,Park,American Restaurant,Bakery,Pub
3,West Toronto,Bar,Café,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Pizza Place,Yoga Studio,Asian Restaurant,Bookstore


# Cluster Boroughs

In [33]:
#set number of clusters
kclusters = 2

Toronto_grouped_clustering = Toronto_grouped.drop('Borough',1)

#run k-means
kmeans = KMeans(n_clusters=kclusters, random_state = 0).fit(Toronto_grouped_clustering)
kmeans.labels_[0:10]



array([1, 0, 0, 0], dtype=int32)

In [34]:
Borough_venues_sorted.insert(0, 'Cluster Lables', kmeans.labels_)

Toronto_merged = df_pf_geo_tor

Toronto_merged = Toronto_merged.join(Borough_venues_sorted.set_index('Borough'), on='Borough')

Toronto_merged.head()

,Postecode,Borough,Neighborhood,Latitude,Logitude,Cluster Lables,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Café,Park,American Restaurant,Bakery,Pub
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Café,Park,American Restaurant,Bakery,Pub
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Café,Park,American Restaurant,Bakery,Pub
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Café,Park,American Restaurant,Bakery,Pub
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Coffee Shop,Park,Sandwich Place,Café,Restaurant,Sushi Restaurant,Pizza Place,Dessert Shop,Clothing Store,Pub


# Visualize clusters 

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = mp.cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [mp.colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Logitude'], Toronto_merged['Borough'], Toronto_merged['Cluster Lables']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Upon Examing the map we can see what constitues most of the clusters in the boroughs of Toronto
Cluster 0: Primarly Greek or coffee shops which are seen in east downtown west Toronto
Cluster 1: shows more of a devation with central Toronto haveing more parks